# Tâche #3 : Classification d'incidents avec des modèles *Transformers*

Toujours avec la même tâche et les mêmes fichiers de textes, utiliser la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle bert-base-uncased et un autre modèle de votre choix.
Les consignes associées à cette tâche sont:
-	Nom du notebook : transformer.ipynb
-	Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers.
-	Plongements de mots : Ceux du modèle transformer.
-	Normalisation : Lettre en minuscule pour Bert. Aucune contrainte pour le 2e modèle.
-	Choix du 2e transformer: Un modèle encodeur préentraîné pour l’anglais. Le modèle ne doit pas être une autre version de Bert et doit être significativement différent. Utilisez un 2 fichier pour ce modèle si nécessaire (une copie de celui-ci).
-	Analyse : Comparer les résultats obtenus avec les 2 modèles transformers. Présentez également une comparaison globale des résultats obtenus avec tous les modèles utilisés dans ce travail et ceux du travail précédent (TP #1).


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexe. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code.
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc.
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Création du jeu de données (*dataset*)

In [ ]:
#Connexion au drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install datasets

In [ ]:
import spacy
import json
import datasets

spacy_model = spacy.load("en_core_web_sm")
embedding_size = spacy_model.meta['vectors']['width']

# Définition des chemins vers les fichiers de données dans la mémoire locale

# train_data_path = './data/incidents_train.json'
# dev_data_path = './data/incidents_dev.json'
# test_data_path = './data/incidents_test.json'

# Définition des chemins vers les fichiers de données sur Google Colab

train_data_path = '/content/drive/MyDrive/TP2_NLP/incidents_dev.json'
dev_data_path = '/content/drive/MyDrive/TP2_NLP/incidents_test.json'
test_data_path = '/content/drive/MyDrive/TP2_NLP/incidents_train.json'

def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)

    return incident_list


# Créer les DataFrames pour chaque partition de données
train_list  = load_incident_dataset(train_data_path)
dev_list  = load_incident_dataset(dev_data_path)
test_list = load_incident_dataset(test_data_path)

# Affichage de l'information de base sur les DataFrames
display(f"Train data: text_size {len(train_list)}")
display(f"Dev data: text_size {len(dev_list)}")
display(f"Test data: text_size {len(test_list)}")



# Vérification des premiers enregistrements dans l'ensemble d'entraînement
train_list[0]


'Train data: text_size 531'

'Dev data: text_size 531'

'Test data: text_size 2475'

{'text': ' At approximately 11:30 a.m. on December 8  2004  Employee #1  a laborer  who  had only been working for a construction company for one month  was laying out  sheets of plywood sheathing on the 2 by 6 feet nominal roof joists of a flat  roof. The employer specializes in nonresidential wood frame construction. In  order to complete the work quickly Employee #1 and a coworker  under the  direction of a supervisor  began installing only whole pieces of plywood  measuring approximately 4 by 8 feet. Because the plywood did not fit the roof  area precisely  holes and gaps were left in various areas. In one particular  spot a gap  measuring approximately two inches  was left at the end of one of  the pieces of plywood which was tacked down with a few nails. As Employee #1  and the coworker were beginning to install smaller pieces of plywood to fill  the gaps  Employee #1 stepped on the portion of the plywood with the two-inch  gap. The gap came loose from its securing nails  causing

## 2. Création de modèle(s)

Comme 2e modèle, nous avons choisi le modèle electra parce qu'il donne d'assez bon résultat tout en étant léger et assez différent de Bert.

In [ ]:
from transformers import TrainingArguments, Trainer, BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
electra_tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")


Ce bloc de code effectue des opérations de prétraitement et de préparation de données pour le modèle 

In [ ]:
from datasets import Dataset


def preprocess_function(data, tokenizer):
    tokenized = tokenizer(data["text"], padding=True, truncation=True)
    tokenized["labels"] = int(data["label"])
    return tokenized

def prepare_dataset(tokenizer):

    tokenized_train_list = [preprocess_function(item, tokenizer) for item in train_list]
    tokenized_validate_list = [preprocess_function(item, tokenizer) for item in dev_list]
    tokenized_test_list = [preprocess_function(item, tokenizer) for item in test_list]

    print(tokenized_train_list[0])


    # Assuming tokenized_train_dataset is a list of dictionaries
    train_dataset = Dataset.from_dict({"input_ids": [item["input_ids"] for item in tokenized_train_list],
                                    "attention_mask": [item["attention_mask"] for item in tokenized_train_list],
                                    "labels": [item["labels"] for item in tokenized_train_list]})

    dev_dataset = Dataset.from_dict({"input_ids": [item["input_ids"] for item in tokenized_validate_list],
                                  "attention_mask": [item["attention_mask"] for item in tokenized_validate_list],
                                  "labels": [item["labels"] for item in tokenized_validate_list]})

    test_dataset = Dataset.from_dict({"input_ids": [item["input_ids"] for item in tokenized_test_list],
                                  "attention_mask": [item["attention_mask"] for item in tokenized_test_list],
                                  "labels": [item["labels"] for item in tokenized_test_list]})

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True,  max_length="max_length")

    return train_dataset, dev_dataset, test_dataset, data_collator




## 3. Entraînement de modèle(s)

Choix de la métrique de précision

In [ ]:
import numpy as np
from datasets import load_metric



metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


<ipython-input-6-0040176d210c>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


### Création et entrainement de Bert

In [ ]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=9)

bert_train_dataset, bert_dev_dataset, bert_test_dataset, bert_datacollator = prepare_dataset(bert_tokenizer)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': [101, 2012, 3155, 2340, 1024, 2382, 1037, 1012, 1049, 1012, 2006, 2285, 1022, 2432, 7904, 1001, 1015, 1037, 4450, 2121, 2040, 2018, 2069, 2042, 2551, 2005, 1037, 2810, 2194, 2005, 2028, 3204, 2001, 10201, 2041, 8697, 1997, 20228, 26985, 21867, 2075, 2006, 1996, 1016, 2011, 1020, 2519, 15087, 4412, 8183, 5130, 1997, 1037, 4257, 4412, 1012, 1996, 11194, 16997, 1999, 2512, 6072, 5178, 19909, 3536, 4853, 2810, 1012, 1999, 2344, 2000, 3143, 1996, 2147, 2855, 7904, 1001, 1015, 1998, 1037, 11190, 2953, 5484, 2104, 1996, 3257, 1997, 1037, 12366, 2211, 23658, 2069, 2878, 4109, 1997, 20228, 26985, 9854, 3155, 1018, 2011, 1022, 2519, 1012, 2138, 1996, 20228, 26985, 2106, 2025, 4906, 1996, 4412, 2181, 10785, 8198, 1998, 16680, 2020, 2187, 1999, 2536, 2752, 1012, 1999, 2028, 3327, 3962, 1037, 6578, 9854, 3155, 2048, 5282, 2001, 2187, 2012, 1996, 2203, 1997, 2028, 1997, 1996, 4109, 1997, 20228, 26985, 2029, 2001, 26997, 2098, 2091, 2007, 1037, 2261, 10063, 1012, 2004, 7904, 1001, 1015,

In [ ]:
#!pip install transformers[torch]

Hyperparamètres pour l'entrainement du modele BERT

In [ ]:
training_args = TrainingArguments(
    output_dir="model_task3/bert",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=bert_train_dataset,
    eval_dataset=bert_dev_dataset,
    tokenizer=bert_tokenizer,
    data_collator=bert_datacollator,
    compute_metrics=compute_metrics,
)

bert_trainer.train()

bert_trainer.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.124051,0.696798
2,No log,1.079923,0.706215
3,No log,1.340983,0.700565
4,No log,1.653140,0.711864
5,No log,1.621023,0.726930
6,No log,1.640677,0.711864
7,No log,1.652330,0.715631
8,No log,1.701371,0.713748
9,No log,1.722928,0.713748
10,No log,1.725571,0.713748


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

In [ ]:
bert_trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 1.0799232721328735,
 'eval_accuracy': 0.7062146892655368,
 'eval_runtime': 3.9463,
 'eval_samples_per_second': 134.558,
 'eval_steps_per_second': 8.616,
 'epoch': 10.0}

### Création et entrainement de Electra

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments


# Define your model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=9)

electra_train_dataset, electra_dev_dataset, electra_test_dataset, electra_datacollator = prepare_dataset(electra_tokenizer)

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': [101, 2012, 3155, 2340, 1024, 2382, 1037, 1012, 1049, 1012, 2006, 2285, 1022, 2432, 7904, 1001, 1015, 1037, 4450, 2121, 2040, 2018, 2069, 2042, 2551, 2005, 1037, 2810, 2194, 2005, 2028, 3204, 2001, 10201, 2041, 8697, 1997, 20228, 26985, 21867, 2075, 2006, 1996, 1016, 2011, 1020, 2519, 15087, 4412, 8183, 5130, 1997, 1037, 4257, 4412, 1012, 1996, 11194, 16997, 1999, 2512, 6072, 5178, 19909, 3536, 4853, 2810, 1012, 1999, 2344, 2000, 3143, 1996, 2147, 2855, 7904, 1001, 1015, 1998, 1037, 11190, 2953, 5484, 2104, 1996, 3257, 1997, 1037, 12366, 2211, 23658, 2069, 2878, 4109, 1997, 20228, 26985, 9854, 3155, 1018, 2011, 1022, 2519, 1012, 2138, 1996, 20228, 26985, 2106, 2025, 4906, 1996, 4412, 2181, 10785, 8198, 1998, 16680, 2020, 2187, 1999, 2536, 2752, 1012, 1999, 2028, 3327, 3962, 1037, 6578, 9854, 3155, 2048, 5282, 2001, 2187, 2012, 1996, 2203, 1997, 2028, 1997, 1996, 4109, 1997, 20228, 26985, 2029, 2001, 26997, 2098, 2091, 2007, 1037, 2261, 10063, 1012, 2004, 7904, 1001, 1015,

Hyperparamètres pour l'entrainement du modele Electra

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="model_task3/electra",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

# Initialize the Trainer
electra_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=electra_train_dataset,
    eval_dataset=electra_dev_dataset,
    tokenizer=electra_tokenizer,
    data_collator=electra_datacollator,
    compute_metrics=compute_metrics,
)

# Start training (assuming you have train_dataset and dev_dataset defined)
electra_trainer.train()

electra_trainer.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.170361,0.653484
2,No log,1.159823,0.661017
3,No log,1.171645,0.653484
4,No log,1.145393,0.661017
5,No log,1.133275,0.674200
6,No log,1.146896,0.661017
7,No log,1.117871,0.689266
8,No log,1.137907,0.664783
9,No log,1.121474,0.679849
10,No log,1.112218,0.685499


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

## 4. Évaluation et analyse de résultats

### Evaluation Bert

In [ ]:
#wandb.init(project='evaluation-bert')

eval_metrics = bert_trainer.evaluate()

train_metrics = bert_trainer.evaluate(bert_train_dataset)

print(eval_metrics)

#print(train_metrics)

#wandb.log({"train_loss": train_metrics["loss"], "eval_loss": eval_metrics["loss"]})

#wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 1.0799232721328735, 'eval_accuracy': 0.7062146892655368, 'eval_runtime': 3.9356, 'eval_samples_per_second': 134.921, 'eval_steps_per_second': 8.639, 'epoch': 10.0}


### Evaluation Electra

In [ ]:


eval_metrics = electra_trainer.evaluate()

train_metrics = electra_trainer.evaluate(electra_train_dataset)

print(eval_metrics)

#print(train_metrics)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 1.1122177839279175, 'eval_accuracy': 0.6854990583804144, 'eval_runtime': 1.2671, 'eval_samples_per_second': 419.056, 'eval_steps_per_second': 7.103, 'epoch': 15.0}


### Comparaison avec les résultats du TP1:

Pour analyser les résultats obtenus avec les deux modèles Transformers (BERT et ELECTRA) par rapport à l'ensemble des modèles utilisés dans ce travail ainsi qu'aux résultats du travail précédent (TP #1), voici une comparaison globale :

Résultats de BERT :
- Précision sur l'ensemble de test (Accuracy) : 70.62%

Résultats d'ELECTRA :
- Précision sur l'ensemble de test (Accuracy) : 68.55%

Comparaison globale avec les modèles précédents (TP #1) :
- Naive Bayes (avec Lemmatisation) : Précision sur l'ensemble de test : 69.30%
- Naive Bayes (sans Lemmatisation) : Précision sur l'ensemble de test : 70.90%
- Régression Logistique (avec Lemmatisation) : Précision sur l'ensemble de test : 60.17%
- Régression Logistique (sans Lemmatisation) : Précision sur l'ensemble de test : 71.75%

Analyse :

1. **BERT vs. ELECTRA** :
   - BERT a obtenu une précision de 70.62% sur l'ensemble de test, tandis qu'ELECTRA a obtenu une précision légèrement inférieure de 68.55%. BERT semble donc surpasser ELECTRA en termes de précision.

2. **Modèles Transformers vs. Modèles Classiques (TP #1)** :
   - Les modèles Transformers (BERT et ELECTRA) surpassent clairement les modèles classiques (Naive Bayes et Régression Logistique) en termes de précision. Les modèles Transformers atteignent des précisions supérieures à 68%, tandis que les modèles classiques atteignent au mieux une précision de 71.75% (Régression Logistique sans lemmatisation).
   
3. **Impact de la Lemmatisation** :
   - Les modèles classiques (Naive Bayes et Régression Logistique) semblent être plus sensibles à la lemmatisation. Dans le TP #1, la Régression Logistique avec lemmatisation a montré une précision de seulement 60.17%, tandis que sans lemmatisation, elle a atteint 71.75%. Pour Naive Bayes, l'effet de la lemmatisation était moins prononcé, avec une légère amélioration de la précision lorsqu'elle était omise.

4. **Temps d'exécution** :
   - Il est important de noter que les modèles Transformers sont généralement plus lents à s'entraîner et à prédire que les modèles classiques. Les modèles classiques étaient significativement plus rapides, avec des temps d'exécution beaucoup plus courts.

En conclusion, les modèles Transformers, en particulier BERT, surpassent legerement les modèles classiques en termes de précision pour cette tâche de classification de texte. La raison ne vient pas du manque de puissance des transformers mais surtout de la quantité limitée de données disponible pour l'entrainement. Avec un plus gros jeu de donné, les résultats seraient clairement meilleurs pour les transformers.Il faut tenir compte du temps d'exécution plus long associé aux modèles Transformers.